In [ ]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports 
from classical_shadow_1_clifford import ClassicalShadow_1_CLIFFORD
from shadow_protocol import ShadowProtocol

from qiskit import QuantumCircuit
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter, ParameterVector
import qiskit.qasm3
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService
from visualisation import LiveVisualizer
import matplotlib.pyplot as plt
import random

In [ ]:
# Setup Experiment
class Bell_Protocol(ShadowProtocol):

    def get_num_qubits(self) -> int:
        return 2
    
    def get_state_circuit(self) -> QuantumCircuit:
        circuit = QuantumCircuit(2)
        #ircuit.h(0)
        #circuit.cx(0,1)
        return circuit

    def run_circuit_and_get_measurement(self, circuit) -> list[int]:
        sim = AerSimulator()

        # Run with 997 shots
        job = sim.run(circuit, shots=997)
        result = job.result()

        counts = result.get_counts()
        max_hits= max(counts, key=counts.get)
        bit_list =  [int(bit) for bit in list(max_hits)]
        return bit_list[::-1]  # Reverse to match Qiskit's qubit ordering
    
bell_protocol = Bell_Protocol()
# Create Classical Shadow Instance
bell_shadow = ClassicalShadow_1_CLIFFORD(bell_protocol)

In [ ]:
target_density_matrix = bell_shadow.get_original_density_matrix()
vis = LiveVisualizer(target_density_matrix)


while True:
    bell_shadow.add_snapshot(log=False)
    
    print(f"Shadow Size: {bell_shadow.get_shadow_size()}")

    actual_density_matrix = bell_shadow.get_density_matrix_from_stabilizers(log=False)
    
    vis.update(actual_density_matrix,bell_shadow.get_shadow_size())

In [ ]:
# Setup Experiment
class SuperPos_Protocol(ShadowProtocol):

    def get_num_qubits(self) -> int:
        return 6
    
    def get_state_circuit(self) -> QuantumCircuit:
        circuit = QuantumCircuit(6)
        circuit.h(0)
        circuit.cx(0, 1)
        circuit.h(2)
        circuit.cx(2, 3)
        circuit.h(4)
        circuit.cx(4, 5)
        return circuit

    def run_circuit_and_get_measurement(self, circuit) -> list[int]:
        sim = AerSimulator()

        # Run with 997 shots
        job = sim.run(circuit, shots=997)
        result = job.result()

        counts = result.get_counts()
        max_hits= max(counts, key=counts.get)
        bit_list = [int(bit) for bit in list(max_hits)]
        return bit_list[::-1]  # Reverse to match Qiskit's qubit ordering
    
superpos_protocol = SuperPos_Protocol()
# Create Classical Shadow Instance
superpos_shadow = ClassicalShadow_1_CLIFFORD(superpos_protocol)

In [ ]:
target_density_matrix = superpos_shadow.get_original_density_matrix()
vis = LiveVisualizer(target_density_matrix)


while True:
    superpos_shadow.add_snapshot()
    
    actual_density_matrix = superpos_shadow.get_density_matrix_from_stabilizers()
    
    vis.update(actual_density_matrix,superpos_shadow.get_shadow_size()) 

In [ ]:
from qiskit import QuantumCircuit, transpile

circuit = QuantumCircuit(2)
circuit.h(0)
qasm_string = qiskit.qasm2.dumps(circuit)
print(qasm_string)
clean_circuit = circuit.copy()
clean_circuit.remove_final_measurements()
qasm_string = qiskit.qasm2.dumps(clean_circuit)
print(qasm_string)
clean_circuit.save_statevector()
# Transpile for simulator
simulator = AerSimulator(method="statevector")
circ = transpile(clean_circuit, simulator)
## Run and get statevector
result = simulator.run(circ).result()
statevector = result.get_statevector(circ)
display(statevector.draw('latex'))


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import DensityMatrix
circuit = QuantumCircuit(2)
circuit.h(0)

clean_circuit = circuit.copy()
clean_circuit.remove_final_measurements()
rho = DensityMatrix(clean_circuit)
print(rho.reverse_qargs().data)